# Project: Face Generation Using Hierarchical VAEs

In [1]:

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

# Load CelebA dataset (make sure it's downloaded and available)
# TensorFlow has a dataset API for CelebA (alternatively, you can use other sources)
dataset = tf.keras.datasets.celeb_a.load_data()

# Preprocess data: Normalize and resize images to 64x64 pixels
image_size = (64, 64)
def preprocess_data(images):
    images = tf.image.resize(images, image_size)  # Resize to 64x64
    images = images / 255.0  # Normalize pixel values to [0, 1]
    return images

train_images = preprocess_data(dataset[0][0])  # Use training data
val_images = preprocess_data(dataset[1][0])  # Use validation data

# Visualize sample images
plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.imshow(train_images[i])
    plt.axis('off')
plt.show()


AttributeError: module 'keras.api.datasets' has no attribute 'celeb_a'

In [ ]:

from tensorflow.keras.layers import Input, Dense, Flatten, Reshape, Conv2D, Conv2DTranspose
from tensorflow.keras.models import Model

# Encoder - hierarchical structure
def create_encoder():
    inputs = Input(shape=(64, 64, 3))
    
    # First level of encoding (low-level features)
    x = Conv2D(32, (3, 3), strides=2, activation='relu')(inputs)
    x = Conv2D(64, (3, 3), strides=2, activation='relu')(x)
    x = Flatten()(x)
    
    # Second level of encoding (higher-level features)
    z1 = Dense(256, activation='relu')(x)
    z2 = Dense(256, activation='relu')(x)
    
    # Final latent variables (hierarchical latent layers)
    z1_mean = Dense(128)(z1)  # Latent variable 1
    z2_mean = Dense(128)(z2)  # Latent variable 2

    model = Model(inputs, [z1_mean, z2_mean], name="encoder")
    return model

# Decoder - generating the image from latent variables
def create_decoder():
    latent_inputs = Input(shape=(128,))
    
    # Combining the hierarchical latent variables
    x = Dense(256, activation='relu')(latent_inputs)
    x = Dense(8*8*128, activation='relu')(x)
    x = Reshape((8, 8, 128))(x)
    
    # Deconvolutional layers (upsampling to image size)
    x = Conv2DTranspose(64, (3, 3), strides=2, activation='relu')(x)
    x = Conv2DTranspose(32, (3, 3), strides=2, activation='relu')(x)
    outputs = Conv2DTranspose(3, (3, 3), activation='sigmoid')(x)  # Output image (3 channels for RGB)

    model = Model(latent_inputs, outputs, name="decoder")
    return model

# Build and compile HVAE model
encoder = create_encoder()
decoder = create_decoder()

# Latent variable input (we take both z1_mean and z2_mean)
latent_inputs = Input(shape=(128,))
generated_image = decoder(latent_inputs)
hvae = Model(latent_inputs, generated_image)

hvae.compile(optimizer='adam', loss='binary_crossentropy')


In [ ]:

# Training HVAE
hvae.fit(train_images, train_images, epochs=20, batch_size=64, validation_data=(val_images, val_images))

# Plot the loss curve
plt.plot(hvae.history.history['loss'], label='Training loss')
plt.plot(hvae.history.history['val_loss'], label='Validation loss')
plt.legend()
plt.title('Training and Validation Loss')
plt.show()


In [ ]:

# Sample new latent variables (z1_mean and z2_mean)
latent_samples = np.random.normal(size=(10, 128))  # Generate 10 samples from latent space

# Generate images from the latent variables
generated_faces = decoder.predict(latent_samples)

# Visualize generated faces
plt.figure(figsize=(10, 10))
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow(generated_faces[i])
    plt.axis('off')
plt.show()
